# Baseline Experiment - Santa 2025 Christmas Tree Packing

## Goals:
1. Load and validate sample_submission.csv
2. Calculate baseline score
3. Set up C++ optimizer (bbox3.cpp)
4. Run optimization and generate improved submission

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def make_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg degrees."""
    p = Polygon(zip(TX, TY))
    p = affinity.rotate(p, deg, origin=(0, 0))
    p = affinity.translate(p, x, y)
    return p

def load_configuration(df, n):
    """Load configuration for n trees from dataframe."""
    prefix = f"{n:03d}_"
    group = df[df['id'].str.startswith(prefix)].copy()
    
    trees = []
    for _, row in group.iterrows():
        x = float(str(row['x'])[1:])  # Strip 's' prefix
        y = float(str(row['y'])[1:])
        deg = float(str(row['deg'])[1:])
        trees.append({'x': x, 'y': y, 'deg': deg, 'polygon': make_tree_polygon(x, y, deg)})
    return trees

def get_bounding_box_side(trees):
    """Get the side length of the bounding box for a configuration."""
    if not trees:
        return 0
    
    all_x = []
    all_y = []
    for tree in trees:
        coords = list(tree['polygon'].exterior.coords)
        all_x.extend([c[0] for c in coords])
        all_y.extend([c[1] for c in coords])
    
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def check_overlaps(trees):
    """Check if any trees overlap (touching is OK)."""
    polygons = [t['polygon'] for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        candidates = tree_index.query(poly)
        for idx in candidates:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    return True
    return False

def calculate_score(df, max_n=200):
    """Calculate total score for submission."""
    total_score = 0
    for n in range(1, max_n + 1):
        trees = load_configuration(df, n)
        if trees:
            side = get_bounding_box_side(trees)
            score = side ** 2 / n
            total_score += score
    return total_score

print("Functions defined successfully!")

Functions defined successfully!


In [2]:
# Load sample submission and calculate baseline score
df = pd.read_csv('/home/data/sample_submission.csv')
print(f"Loaded {len(df)} rows")
print(f"Sample rows:")
print(df.head(10))

Loaded 20100 rows
Sample rows:
      id           x           y     deg
0  001_0        s0.0        s0.0   s90.0
1  002_0        s0.0        s0.0   s90.0
2  002_1   s0.202736  s-0.511271   s90.0
3  003_0        s0.0        s0.0   s90.0
4  003_1   s0.202736  s-0.511271   s90.0
5  003_2     s0.5206   s0.177413  s180.0
6  004_0        s0.0        s0.0   s90.0
7  004_1   s0.202736  s-0.511271   s90.0
8  004_2     s0.5206   s0.177413  s180.0
9  004_3  s-0.818657  s-0.228694  s180.0


In [3]:
# Calculate score for sample submission
print("Calculating baseline score (this may take a minute)...")
baseline_score = calculate_score(df)
print(f"\nBaseline Score: {baseline_score:.6f}")

Calculating baseline score (this may take a minute)...



Baseline Score: 173.652299


In [4]:
# Check for overlaps in a few configurations
print("Checking for overlaps in sample configurations...")
for n in [1, 10, 50, 100, 200]:
    trees = load_configuration(df, n)
    has_overlap = check_overlaps(trees)
    side = get_bounding_box_side(trees)
    print(f"N={n:3d}: side={side:.4f}, score_contrib={side**2/n:.4f}, overlaps={has_overlap}")

Checking for overlaps in sample configurations...
N=  1: side=1.0000, score_contrib=1.0000, overlaps=False
N= 10: side=3.4411, score_contrib=1.1841, overlaps=False
N= 50: side=6.6492, score_contrib=0.8842, overlaps=False
N=100: side=8.9561, score_contrib=0.8021, overlaps=False
N=200: side=13.0345, score_contrib=0.8495, overlaps=False


In [5]:
# Copy sample submission as our starting baseline
import shutil
shutil.copy('/home/data/sample_submission.csv', '/home/submission/submission.csv')
print("Copied sample_submission.csv to /home/submission/submission.csv")

Copied sample_submission.csv to /home/submission/submission.csv


In [6]:
# Now let's set up the C++ optimizer
# First, extract bbox3.cpp from the kernel

bbox3_cpp = '''// BBOX3 - Global Dynamics Edition
// Features: Complex Number Vector Coordination, Fluid Dynamics, Hinge Pivot, 
// Density Gradient Flow, and NEW Global Boundary Tension.
// Uses a separate 'global_squeeze' function for Dynamic Scaling and Overlap Repair.

#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <string>
#include <vector>
#include <map>
#include <set>
#include <tuple>
#include <iomanip>
#include <chrono>
#include <random>
#include <numeric>
#include <omp.h>
#include <complex> 

using namespace std;
using namespace chrono;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;
constexpr double EPSILON = 1e-16;
constexpr double NEIGHBOR_RADIUS = 0.5;      
constexpr double PIVOT_ANGLE_MAX = 10.0;     
constexpr double GLOBAL_TENSION_STRENGTH = 0.05; 

// Base tree geometry 
const double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
const double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

thread_local mt19937_64 rng(44); 
thread_local uniform_real_distribution<double> U(0, 1);

inline double rf() { return U(rng); }
inline int ri(int n) { return rng() % n; }

// --- Geometric Structures and Functions (Complex Numbers) ---

using Complex = std::complex<double>;

// Forward declarations for mutual dependency
struct Cfg;
struct Poly;

Cfg global_squeeze(Cfg c, uint64_t seed);
Complex getSeparationVector(const Poly& a, const Poly& b); 

struct Poly {
    Complex p[NV]; 
    double x0, y0, x1, y1;
    void bbox() {
        x0 = x1 = p[0].real(); y0 = y1 = p[0].imag();
        for (int i = 1; i < NV; i++) {
            x0 = min(x0, p[i].real()); x1 = max(x1, p[i].real());
            y0 = min(y0, p[i].imag()); y1 = max(y1, p[i].imag());
        }
    }
};

Poly getPoly(Complex c_center, double deg) {
    Poly q;
    double r = deg * PI / 180;
    Complex c_rot = polar(1.0, r); 

    for (int i = 0; i < NV; i++) {
        Complex base_pt(TX[i], TY[i]);
        Complex rotated_pt = base_pt * c_rot; 
        q.p[i] = rotated_pt + c_center;
    }
    q.bbox();
    return q;
}

bool pip(double px, double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        double qi_x = q.p[i].real(), qi_y = q.p[i].imag();
        double qj_x = q.p[j].real(), qj_y = q.p[j].imag();
        if ((qi_y > py) != (qj_y > py) &&
            px < (qj_x - qi_x) * (py - qi_y) / (qj_y - qi_y) + qi_x)
            in = !in;
        j = i;
    }
    return in;
}

bool segInt(Complex a, Complex b, Complex c, Complex d) {
    auto ccw = [](Complex p, Complex q, Complex r) { 
        return (r.imag() - p.imag()) * (q.real() - p.real()) > (q.imag() - p.imag()) * (r.real() - p.real()); 
    };
    return ccw(a, c, d) != ccw(b, c, d) && ccw(a, b, c) != ccw(a, b, d);
}

// Minimal overlap check
bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.p[i].real(), a.p[i].imag(), b)) return true;
        if (pip(b.p[i].real(), b.p[i].imag(), a)) return true;
    }
    for (int i = 0; i < NV; i++)
        for (int j = 0; j < NV; j++)
            if (segInt(a.p[i], a.p[(i + 1) % NV], b.p[j], b.p[(j + 1) % NV])) return true;
    return false;
}

struct Cfg {
    int n;
    Complex c[MAX_N]; 
    double a[MAX_N];  
    Poly pl[MAX_N];

    void upd(int i) { pl[i] = getPoly(c[i], a[i]); }
    void updAll() { for (int i = 0; i < n; i++) upd(i); }

    bool hasOvl(int i) const {
        for (int j = 0; j < n; j++) { 
            if (i != j && overlap(pl[i], pl[j])) {
                return true;
            }
        }
        return false;
    }

    bool hasOvlPair(int i, int j) const {
        if (overlap(pl[i], pl[j])) return true;
        for (int k = 0; k < n; k++) {
            if (k != i && k != j) {
                if (overlap(pl[i], pl[k]) || overlap(pl[j], pl[k])) {
                    return true;
                }
            }
        }
        return false;
    }

    bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }

    double side() const {
        if (!n) return 0;
        double x0 = pl[0].x0, x1 = pl[0].x1, y0 = pl[0].y0, y1 = pl[0].y1;
        for (int i = 1; i < n; i++) {
            x0 = min(x0, pl[i].x0); x1 = max(x1, pl[i].x1);
            y0 = min(y0, pl[i].y0); y1 = max(y1, pl[i].y1);
        }
        return max(x1 - x0, y1 - y0);
    }

    double score() const { double s = side(); return s * s / n; }

    Complex centroid() const {
        Complex sum = 0.0;
        for (int i = 0; i < n; i++) { sum += c[i]; }
        return sum / (double)n;
    }

    tuple<double, double, double, double> getBBox() const {
        double gx0 = pl[0].x0, gx1 = pl[0].x1, gy0 = pl[0].y0, gy1 = pl[0].y1;
        for (int i = 1; i < n; i++) {
            gx0 = min(gx0, pl[i].x0); gx1 = max(gx1, pl[i].x1);
            gy0 = min(gy0, pl[i].y0); gy1 = max(gy1, pl[i].y1);
        }
        return {gx0, gy0, gx1, gy1};
    }

    vector<int> findCornerTrees() const {
        auto [gx0, gy0, gx1, gy1] = getBBox();
        double eps = 0.01;
        vector<int> corners;
        for (int i = 0; i < n; i++) {
            if (abs(pl[i].x0 - gx0) < eps || abs(pl[i].x1 - gx1) < eps ||
                abs(pl[i].y0 - gy0) < eps || abs(pl[i].y1 - gy1) < eps) {
                corners.push_back(i);
            }
        }
        return corners;
    }
};

// --- Overlap Resolution Helper Function ---

Complex getSeparationVector(const Poly& a, const Poly& b) {
    Complex c_a = a.p[0]; for(int i=1; i<NV; ++i) c_a += a.p[i]; c_a /= (double)NV;
    Complex c_b = b.p[0]; for(int i=1; i<NV; ++i) c_b += b.p[i]; c_b /= (double)NV;

    Complex diff = c_a - c_b;
    double dist = abs(diff);
    double overlap_depth = 0.2; 

    if (dist > EPSILON) {
        return diff / dist * overlap_depth;
    } else {
        return Complex(rf() * 0.1, rf() * 0.1);
    }
}

// --- Optimization Routines ---

Cfg aggressive_repair(Cfg c, int max_cycles) {
    Cfg current = c;
    for (int cycle = 0; cycle < max_cycles; ++cycle) {
        bool repaired = true;
        for (int i = 0; i < current.n; ++i) {
            for (int j = i + 1; j < current.n; ++j) {
                if (overlap(current.pl[i], current.pl[j])) {
                    repaired = false;
                    Complex sep_vector = getSeparationVector(current.pl[i], current.pl[j]);
                    current.c[i] += sep_vector * 0.5;
                    current.c[j] -= sep_vector * 0.5;
                    current.upd(i);
                    current.upd(j);
                }
            }
        }
        if (repaired) break;
    }
    return current;
}

Cfg global_squeeze(Cfg c, uint64_t seed) {
    rng.seed(seed);
    Cfg best = c;
    double best_side = c.side();

    for (int iter = 0; iter < 500; ++iter) {
        double scale = 0.995 + rf() * 0.004; // 0.995 to 0.999
        Cfg trial = c;
        Complex cent = trial.centroid();
        
        for (int i = 0; i < trial.n; ++i) {
            trial.c[i] = cent + (trial.c[i] - cent) * scale;
        }
        trial.updAll();
        
        // Repair overlaps
        trial = aggressive_repair(trial, 50);
        
        if (!trial.anyOvl()) {
            double s = trial.side();
            if (s < best_side) {
                best = trial;
                best_side = s;
            }
        }
    }
    return best;
}

Cfg local_search(Cfg c, int iters) {
    const double steps[] = {0.01, 0.004, 0.0015, 0.0006, 0.00025, 0.0001};
    const double angles[] = {5.0, 2.0, 0.8, 0.3, 0.1};
    const int dx[] = {1, -1, 0, 0, 1, 1, -1, -1};
    const int dy[] = {0, 0, 1, -1, 1, -1, 1, -1};
    
    Cfg best = c;
    double best_side = c.side();
    
    for (int it = 0; it < iters; ++it) {
        bool improved = false;
        
        // Focus on corner trees
        vector<int> corners = best.findCornerTrees();
        if (corners.empty()) {
            for (int i = 0; i < best.n; ++i) corners.push_back(i);
        }
        
        for (int idx : corners) {
            Complex orig_c = best.c[idx];
            double orig_a = best.a[idx];
            
            // Try position moves
            for (double step : steps) {
                for (int d = 0; d < 8; ++d) {
                    best.c[idx] = orig_c + Complex(dx[d] * step, dy[d] * step);
                    best.upd(idx);
                    
                    if (!best.hasOvl(idx)) {
                        double s = best.side();
                        if (s < best_side) {
                            best_side = s;
                            orig_c = best.c[idx];
                            improved = true;
                        }
                    }
                    best.c[idx] = orig_c;
                    best.upd(idx);
                }
            }
            
            // Try rotation moves
            for (double ang : angles) {
                for (int sign = -1; sign <= 1; sign += 2) {
                    best.a[idx] = orig_a + sign * ang;
                    best.upd(idx);
                    
                    if (!best.hasOvl(idx)) {
                        double s = best.side();
                        if (s < best_side) {
                            best_side = s;
                            orig_a = best.a[idx];
                            improved = true;
                        }
                    }
                    best.a[idx] = orig_a;
                    best.upd(idx);
                }
            }
        }
        
        if (!improved) break;
    }
    return best;
}

Cfg compaction(Cfg c, int iters) {
    Cfg best = c;
    double best_side = c.side();
    
    for (int it = 0; it < iters; ++it) {
        Complex cent = best.centroid();
        bool improved = false;
        
        for (int i = 0; i < best.n; ++i) {
            Complex orig = best.c[i];
            Complex dir = cent - orig;
            double dist = abs(dir);
            if (dist < EPSILON) continue;
            dir /= dist;
            
            // Move toward center
            for (double step = 0.1; step > 0.001; step *= 0.5) {
                best.c[i] = orig + dir * step;
                best.upd(i);
                
                if (!best.hasOvl(i)) {
                    double s = best.side();
                    if (s < best_side) {
                        best_side = s;
                        orig = best.c[i];
                        improved = true;
                    }
                }
                best.c[i] = orig;
                best.upd(i);
            }
        }
        
        if (!improved) break;
    }
    return best;
}

// --- I/O Functions ---

map<int, Cfg> loadCSV(const string& fname) {
    map<int, Cfg> cfgs;
    ifstream f(fname);
    string line;
    getline(f, line); // header
    
    while (getline(f, line)) {
        stringstream ss(line);
        string id, xs, ys, ds;
        getline(ss, id, ',');
        getline(ss, xs, ',');
        getline(ss, ys, ',');
        getline(ss, ds, ',');
        
        int n = stoi(id.substr(0, 3));
        int idx = stoi(id.substr(4));
        
        double x = stod(xs.substr(1));
        double y = stod(ys.substr(1));
        double d = stod(ds.substr(1));
        
        if (cfgs.find(n) == cfgs.end()) {
            cfgs[n].n = n;
        }
        cfgs[n].c[idx] = Complex(x, y);
        cfgs[n].a[idx] = d;
    }
    
    for (auto& [n, cfg] : cfgs) {
        cfg.updAll();
    }
    return cfgs;
}

void saveCSV(const string& fname, const map<int, Cfg>& cfgs) {
    ofstream f(fname);
    f << "id,x,y,deg\\n";
    f << fixed << setprecision(12);
    
    for (int n = 1; n <= 200; ++n) {
        if (cfgs.find(n) == cfgs.end()) continue;
        const Cfg& cfg = cfgs.at(n);
        for (int i = 0; i < cfg.n; ++i) {
            f << setfill('0') << setw(3) << n << "_" << i << ",";
            f << "s" << cfg.c[i].real() << ",";
            f << "s" << cfg.c[i].imag() << ",";
            f << "s" << cfg.a[i] << "\\n";
        }
    }
}

double totalScore(const map<int, Cfg>& cfgs) {
    double total = 0;
    for (const auto& [n, cfg] : cfgs) {
        total += cfg.score();
    }
    return total;
}

int main(int argc, char* argv[]) {
    string input_file = "submission.csv";
    string output_file = "submission_optimized.csv";
    int n_iters = 100;
    int n_restarts = 10;
    
    for (int i = 1; i < argc; i++) {
        string arg = argv[i];
        if (arg == "-i" && i + 1 < argc) input_file = argv[++i];
        else if (arg == "-o" && i + 1 < argc) output_file = argv[++i];
        else if (arg == "-n" && i + 1 < argc) n_iters = stoi(argv[++i]);
        else if (arg == "-r" && i + 1 < argc) n_restarts = stoi(argv[++i]);
    }
    
    cout << "Loading " << input_file << "..." << endl;
    map<int, Cfg> cfgs = loadCSV(input_file);
    cout << "Initial score: " << totalScore(cfgs) << endl;
    
    auto start = high_resolution_clock::now();
    
    #pragma omp parallel for schedule(dynamic)
    for (int n = 1; n <= 200; ++n) {
        if (cfgs.find(n) == cfgs.end()) continue;
        
        Cfg best = cfgs[n];
        double best_score = best.score();
        
        for (int r = 0; r < n_restarts; ++r) {
            Cfg trial = cfgs[n];
            
            // Apply optimization sequence
            trial = global_squeeze(trial, r * 12345 + n);
            trial = local_search(trial, n_iters);
            trial = compaction(trial, n_iters);
            trial = local_search(trial, n_iters);
            
            if (!trial.anyOvl()) {
                double s = trial.score();
                if (s < best_score) {
                    best = trial;
                    best_score = s;
                }
            }
        }
        
        #pragma omp critical
        {
            cfgs[n] = best;
            if (n % 20 == 0) {
                cout << "Processed N=" << n << ", score=" << best_score << endl;
            }
        }
    }
    
    auto end = high_resolution_clock::now();
    double elapsed = duration_cast<seconds>(end - start).count();
    
    cout << "Final score: " << totalScore(cfgs) << endl;
    cout << "Time: " << elapsed << " seconds" << endl;
    
    saveCSV(output_file, cfgs);
    cout << "Saved to " << output_file << endl;
    
    return 0;
}
'''

with open('/home/code/experiments/001_baseline/bbox3.cpp', 'w') as f:
    f.write(bbox3_cpp)
print("Wrote bbox3.cpp")

Wrote bbox3.cpp


In [7]:
# Compile the C++ optimizer
import subprocess
result = subprocess.run(
    ['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 
     '/home/code/experiments/001_baseline/bbox3', 
     '/home/code/experiments/001_baseline/bbox3.cpp'],
    capture_output=True, text=True
)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
print("Return code:", result.returncode)

STDOUT: 
STDERR: 
Return code: 0


In [8]:
# Copy sample submission to working directory and run optimizer
import shutil
shutil.copy('/home/data/sample_submission.csv', '/home/code/experiments/001_baseline/submission.csv')

# Run the optimizer with moderate settings (quick test)
import subprocess
import time

start = time.time()
result = subprocess.run(
    ['/home/code/experiments/001_baseline/bbox3',
     '-i', '/home/code/experiments/001_baseline/submission.csv',
     '-o', '/home/code/experiments/001_baseline/submission_optimized.csv',
     '-n', '50',  # iterations
     '-r', '5'],  # restarts
    capture_output=True, text=True,
    cwd='/home/code/experiments/001_baseline'
)
elapsed = time.time() - start

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
print(f"Time: {elapsed:.1f} seconds")

STDOUT: Loading /home/code/experiments/001_baseline/submission.csv...
Initial score: 173.652
Processed N=20, score=0.788572
Processed N=40, score=0.834393
Processed N=60, score=0.82306
Processed N=80, score=0.701761
Processed N=100, score=0.783528
Processed N=120, score=0.742152
Processed N=140, score=0.839883
Processed N=160, score=0.786562
Processed N=180, score=0.822553
Processed N=200, score=0.831356
Final score: 164.924
Time: 630 seconds
Saved to /home/code/experiments/001_baseline/submission_optimized.csv

STDERR: 
Time: 630.4 seconds


In [9]:
# Validate the optimized submission and calculate score\nimport pandas as pd\n\ndf_opt = pd.read_csv('/home/code/experiments/001_baseline/submission_optimized.csv')\nprint(f\"Loaded {len(df_opt)} rows\")\n\n# Calculate score\nopt_score = calculate_score(df_opt)\nprint(f\"Optimized Score: {opt_score:.6f}\")\n\n# Check for overlaps in key configurations\nprint(\"\\nChecking for overlaps...\")\nfor n in [1, 10, 50, 100, 200]:\n    trees = load_configuration(df_opt, n)\n    has_overlap = check_overlaps(trees)\n    side = get_bounding_box_side(trees)\n    print(f\"N={n:3d}: side={side:.4f}, score_contrib={side**2/n:.4f}, overlaps={has_overlap}\")

In [10]:
# Implement rotation optimization (fix_direction)\nfrom scipy.spatial import ConvexHull\nfrom scipy.optimize import minimize_scalar\nimport numpy as np\n\ndef calculate_bbox_side_at_angle(angle_deg, points):\n    \"\"\"Calculate bounding box side length at a given rotation angle.\"\"\"\n    angle_rad = np.radians(angle_deg)\n    c, s = np.cos(angle_rad), np.sin(angle_rad)\n    rot_matrix_T = np.array([[c, s], [-s, c]])\n    rotated_points = points.dot(rot_matrix_T)\n    min_xy = np.min(rotated_points, axis=0)\n    max_xy = np.max(rotated_points, axis=0)\n    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])\n\ndef optimize_rotation_for_config(trees):\n    \"\"\"Find optimal rotation angle to minimize bounding box.\"\"\"\n    # Get all polygon vertices\n    all_points = []\n    for tree in trees:\n        coords = list(tree['polygon'].exterior.coords)\n        all_points.extend(coords)\n    points_np = np.array(all_points)\n    \n    # Get convex hull for efficiency\n    try:\n        hull_points = points_np[ConvexHull(points_np).vertices]\n    except:\n        hull_points = points_np\n    \n    initial_side = calculate_bbox_side_at_angle(0, hull_points)\n    \n    # Find optimal rotation angle (0-90 degrees due to symmetry)\n    res = minimize_scalar(\n        lambda a: calculate_bbox_side_at_angle(a, hull_points),\n        bounds=(0.001, 89.999), method='bounded'\n    )\n    \n    found_angle_deg = res.x\n    found_side = res.fun\n    \n    improvement = initial_side - found_side\n    \n    if improvement > 1e-8:\n        return found_angle_deg, found_side\n    else:\n        return 0.0, initial_side\n\ndef apply_rotation_to_config(trees, angle_deg):\n    \"\"\"Apply rotation to all trees in a configuration.\"\"\"\n    if abs(angle_deg) < 1e-9:\n        return trees\n    \n    # Find center of bounding box\n    all_x = []\n    all_y = []\n    for tree in trees:\n        coords = list(tree['polygon'].exterior.coords)\n        all_x.extend([c[0] for c in coords])\n        all_y.extend([c[1] for c in coords])\n    \n    center_x = (min(all_x) + max(all_x)) / 2\n    center_y = (min(all_y) + max(all_y)) / 2\n    \n    angle_rad = np.radians(angle_deg)\n    c, s = np.cos(angle_rad), np.sin(angle_rad)\n    \n    new_trees = []\n    for tree in trees:\n        # Rotate position around center\n        dx = tree['x'] - center_x\n        dy = tree['y'] - center_y\n        new_x = center_x + dx * c - dy * s\n        new_y = center_y + dx * s + dy * c\n        new_deg = tree['deg'] + angle_deg\n        \n        new_trees.append({\n            'x': new_x,\n            'y': new_y,\n            'deg': new_deg,\n            'polygon': make_tree_polygon(new_x, new_y, new_deg)\n        })\n    \n    return new_trees\n\nprint(\"Rotation optimization functions defined!\")

In [11]:
# Apply rotation optimization to the optimized submission\nprint(\"Applying rotation optimization to all configurations...\")\n\ndf_opt = pd.read_csv('/home/code/experiments/001_baseline/submission_optimized.csv')\n\nimproved_configs = {}\ntotal_improvement = 0\n\nfor n in range(1, 201):\n    trees = load_configuration(df_opt, n)\n    if not trees:\n        continue\n    \n    old_side = get_bounding_box_side(trees)\n    old_score = old_side ** 2 / n\n    \n    # Find optimal rotation\n    best_angle, new_side = optimize_rotation_for_config(trees)\n    \n    if best_angle > 0.01:  # Only apply if meaningful rotation found\n        # Apply rotation\n        rotated_trees = apply_rotation_to_config(trees, best_angle)\n        \n        # Verify no overlaps\n        if not check_overlaps(rotated_trees):\n            new_score = new_side ** 2 / n\n            improvement = old_score - new_score\n            \n            if improvement > 1e-10:\n                improved_configs[n] = rotated_trees\n                total_improvement += improvement\n                if n % 20 == 0:\n                    print(f\"N={n}: angle={best_angle:.2f}°, improvement={improvement:.6f}\")\n\nprint(f\"\\nTotal improvement from rotation: {total_improvement:.6f}\")\nprint(f\"Configurations improved: {len(improved_configs)}\")

In [12]:
# Save the improved submission with rotation optimization\nprint(f\"Improved configs: {len(improved_configs)}\")\nprint(f\"Total improvement: {total_improvement:.6f}\")\n\n# Create final submission by updating the optimized file with rotated configs\ndf_final = df_opt.copy()\n\nfor n, trees in improved_configs.items():\n    prefix = f\"{n:03d}_\"\n    mask = df_final['id'].str.startswith(prefix)\n    \n    for i, tree in enumerate(trees):\n        row_id = f\"{n:03d}_{i}\"\n        idx = df_final[df_final['id'] == row_id].index[0]\n        df_final.loc[idx, 'x'] = f\"s{tree['x']:.12f}\"\n        df_final.loc[idx, 'y'] = f\"s{tree['y']:.12f}\"\n        df_final.loc[idx, 'deg'] = f\"s{tree['deg']:.12f}\"\n\n# Calculate final score\nfinal_score = calculate_score(df_final)\nprint(f\"\\nFinal Score after rotation optimization: {final_score:.6f}\")\n\n# Save to submission folder\ndf_final.to_csv('/home/submission/submission.csv', index=False)\nprint(\"Saved to /home/submission/submission.csv\")

In [13]:
# Run a longer optimization with more restarts\nimport subprocess\nimport time\nimport shutil\n\n# Use the optimized submission as input for further optimization\nshutil.copy('/home/code/experiments/001_baseline/submission_optimized.csv', \n            '/home/code/experiments/001_baseline/submission_v2.csv')\n\nprint(\"Running extended optimization (this will take several minutes)...\")\nstart = time.time()\n\nresult = subprocess.run(\n    ['/home/code/experiments/001_baseline/bbox3',\n     '-i', '/home/code/experiments/001_baseline/submission_v2.csv',\n     '-o', '/home/code/experiments/001_baseline/submission_v3.csv',\n     '-n', '200',  # more iterations\n     '-r', '20'],  # more restarts\n    capture_output=True, text=True,\n    cwd='/home/code/experiments/001_baseline',\n    timeout=1800  # 30 minute timeout\n)\n\nelapsed = time.time() - start\nprint(f\"Time: {elapsed:.1f} seconds\")\nprint(\"STDOUT:\", result.stdout[-2000:] if len(result.stdout) > 2000 else result.stdout)

In [14]:
# Implement a better greedy initialization based on the getting-started kernel\n# Key insight: Use weighted random angles favoring diagonals, and try multiple restarts\n\nimport numpy as np\nfrom shapely.geometry import Polygon\nfrom shapely import affinity\nfrom shapely.strtree import STRtree\nimport random\n\nTX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]\nTY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]\n\ndef make_tree(x, y, deg):\n    p = Polygon(zip(TX, TY))\n    p = affinity.rotate(p, deg, origin=(0, 0))\n    p = affinity.translate(p, x, y)\n    return {'x': x, 'y': y, 'deg': deg, 'polygon': p}\n\ndef check_collision(tree, existing_trees):\n    \"\"\"Check if tree collides with any existing trees.\"\"\"\n    for t in existing_trees:\n        if tree['polygon'].intersects(t['polygon']) and not tree['polygon'].touches(t['polygon']):\n            return True\n    return False\n\ndef get_bbox_side(trees):\n    if not trees:\n        return 0\n    all_x, all_y = [], []\n    for t in trees:\n        coords = list(t['polygon'].exterior.coords)\n        all_x.extend([c[0] for c in coords])\n        all_y.extend([c[1] for c in coords])\n    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))\n\ndef greedy_pack(n, num_attempts=10):\n    \"\"\"Pack n trees using greedy approach with multiple attempts.\"\"\"\n    best_trees = None\n    best_side = float('inf')\n    \n    for attempt in range(num_attempts):\n        trees = []\n        \n        for i in range(n):\n            placed = False\n            \n            # Try multiple starting angles\n            for _ in range(50):\n                # Weighted random angle favoring diagonals\n                angle = random.uniform(0, 360)\n                weight = abs(np.sin(2 * np.radians(angle)))\n                if random.random() > weight:\n                    continue\n                \n                # Start far from center and move in\n                for deg in [0, 90, 180, 270]:\n                    radius = 20.0\n                    x = radius * np.cos(np.radians(angle))\n                    y = radius * np.sin(np.radians(angle))\n                    \n                    # Move toward center\n                    step = 0.5\n                    while radius > 0:\n                        tree = make_tree(x, y, deg)\n                        if not check_collision(tree, trees):\n                            # Found valid position, now move closer\n                            last_valid = tree\n                            while True:\n                                radius -= step\n                                if radius < 0:\n                                    break\n                                x = radius * np.cos(np.radians(angle))\n                                y = radius * np.sin(np.radians(angle))\n                                tree = make_tree(x, y, deg)\n                                if check_collision(tree, trees):\n                                    break\n                                last_valid = tree\n                            trees.append(last_valid)\n                            placed = True\n                            break\n                        radius -= step\n                        x = radius * np.cos(np.radians(angle))\n                        y = radius * np.sin(np.radians(angle))\n                    \n                    if placed:\n                        break\n                if placed:\n                    break\n            \n            if not placed:\n                # Fallback: place at a random position\n                for _ in range(100):\n                    x = random.uniform(-10, 10)\n                    y = random.uniform(-10, 10)\n                    deg = random.choice([0, 90, 180, 270])\n                    tree = make_tree(x, y, deg)\n                    if not check_collision(tree, trees):\n                        trees.append(tree)\n                        placed = True\n                        break\n        \n        if len(trees) == n:\n            side = get_bbox_side(trees)\n            if side < best_side:\n                best_side = side\n                best_trees = trees\n    \n    return best_trees, best_side\n\nprint(\"Greedy packing functions defined!\")\n\n# Test on small n\ntrees, side = greedy_pack(10, num_attempts=5)\nprint(f\"N=10: side={side:.4f}, score={side**2/10:.4f}\")